<a href="https://colab.research.google.com/github/R4INYIS/Logic-Project/blob/main/Proyecto_l%C3%B3gica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 1


In [ ]:
def evaluarForma(forma: str, lista: list, asignacion: set) -> bool:
  """
  Dado una fórmula y su forma, evalua si la fórmula es verdadera o falsa
  """
  asignacion['Top'], asignacion['Bot'] = 1, 0
  if forma == 'FNC':
    for clausula in lista:
      clausula_verdadera = None
      for literal in clausula:
        literal = '!' * (literal.count('!') % 2) + literal.replace('!', '')
        if literal.startswith('!'):
          valor_literal = not asignacion[literal[-1]]
        else:
          valor_literal = asignacion[literal]
        if valor_literal == 1:
          clausula_verdadera = True
          break

      if clausula_verdadera == None:
        return False
    return True

  elif forma == 'FND':
    for clausula in lista:
      clausula_verdadera = True
      for literal in clausula:
        literal = '!' * (literal.count('!') % 2) + literal.replace('!', '')
        valor_literal = not asignacion[literal[-1]] if literal.startswith('!') else asignacion[literal]
        if valor_literal == 0:
          clausula_verdadera = False
          break
      if clausula_verdadera:
        return True
    return False
  else:
    print('Elige un tipo de forma de los siguientes FNC o FND')

lista = [["!!p", "!q", "r" ],["Bot", "!q"]]
asignacion = {"p": 1, "q": 0, "r": 1}
forma = 'FNC'
evaluarForma(forma, lista, asignacion)

True

In [ ]:
def esTautologıaFNC(lista: list) -> bool:
  """
  Verifica si una lista de cláusulas en formato FNC es una tautología
  """
  if len(lista) == 0:
    return 1
  for clausula in lista:
    clausula_tautologia = False
    if 'Top' in clausula:
      continue
    for literal in clausula:
      literal = '!' * (literal.count('!') % 2) + literal.replace('!', '')
      opuesto = f"!{literal}" if len(literal) == 1 else literal[1]
      if opuesto in clausula:
        clausula_tautologia = True
        break
    if clausula_tautologia == False:
      return False
  return True

lista = [["!q", "!!q", "r" ], ["!r", "r"]]
esTautologıaFNC(lista)

True

# Parte 2


In [7]:
conectores = ['^', '*', '>', '<'] # Orden de precedencia importa para desempates

def aplicarConectores(l1: bool, l2: bool, conector: str) -> bool:
  """
  Verifica el resultado de una fórmula dado un conector y dos literales
  """
  if conector == '^':
    return l1 and l2
  elif conector == '*':
    return l1 or l2
  elif conector == '>':
    return not l1 or l2
  elif conector == '<':
    return l1 == l2

def dividirFormula(formula: str, asignacion: set = {}) -> tuple:
  """
  Dada una fórmula en formato str devuelve una tupla anidada de los conectores y literales
  """
  formula = formula.strip().replace(' ', '')
  # Se verica si los paréntesis se abren y se cierra en toda la fórmula y los elimina para dejar las claúsulas limpias
  while formula.startswith('(') and formula.endswith(')'):
    balance = 0 # Balance de parentesis
    encierra_todo = True
    for char in formula[:-1]: # Eliminamos el último paréntesis, esto nos permite afirmar que si el balance es 0, hay algún elemento que no está entre paréntesis
      if char == '(':
        balance += 1
      elif char == ')':
        balance -= 1
        if balance == 0:
          encierra_todo = False
          break
    if encierra_todo:
      formula = formula[1:-1].strip()
    else:
      break

  balance = 0
  indice_conector_principal = None
  min_balance = float('inf') # Inicializamos la variable en el nivel infinito para cualquier conector presente este en una capa más alta

  for i, caracter in enumerate(formula):
    if caracter == '(':
      balance += 1
    elif caracter == ')':
      balance -= 1
    elif caracter in conectores:
      if balance < min_balance: # Si el conector esta dentro de menos paréntesis que el anterior se vuelve el nuevo conector principal
        min_balance = balance
        indice_conector_principal = i

      elif balance == min_balance:
        indice_conector_anterior = conectores.index(formula[indice_conector_principal])
        indice_actual = conectores.index(caracter)
        if indice_actual > indice_conector_anterior: # Si el conector actual tiene mayor precedencia
          indice_conector_principal = i

  if not indice_conector_principal: # Es un literal
    return formula.replace('(', '').replace(')', '')

  conector = formula[indice_conector_principal]


  # Repetimos el proceso recursivamente con las cláusulas interiores
  clausula_izq = dividirFormula(formula[:indice_conector_principal])
  clausula_der = dividirFormula(formula[indice_conector_principal + 1:])

  # Se devuelve la fórmula/cláusula
  return (clausula_izq, clausula_der, conector)

def verificarFormula(formula: tuple, asignacion: set) -> bool:
  """
  Verifica si dados unos valores una fórmula booleana (str) es verdadera o no
  """
  if isinstance(formula, str):
    return not asignacion[formula[-1]] if formula.startswith('!') else asignacion[formula]

  clausula1 = formula[0]
  clausula2 = formula[1]
  conector_principal = formula[2]

  if isinstance(clausula1, tuple):
    clausula1 = verificarFormula(clausula1, asignacion)

  if isinstance(clausula2, tuple):
    clausula2 = verificarFormula(clausula2, asignacion)

  if isinstance(clausula1, (str, int)) and isinstance(clausula2, (str, int)):
    if isinstance(clausula1, str):
      l1 = not asignacion[clausula1[-1]] if clausula1.startswith('!') else asignacion[clausula1]
    else:
      l1 = clausula1

    if isinstance(clausula2, str):
      l2 = not asignacion[clausula2[-1]] if clausula2.startswith('!') else asignacion[clausula2]
    else:
      l2 = clausula2
    formula = aplicarConectores(l1, l2, conector_principal)


  return formula


def evaluarVerdad(formula: str, asignacion: set) -> bool:
  """
  Verifica si dados unos valores una fórmula booleana (str) es verdadera o no
  """
  formula = dividirFormula(formula)
  valor = verificarFormula(formula, asignacion)

  return bool(valor)




formula_str = '((q) * (!(p)))'
print(evaluarVerdad(formula_str, {'p': 1, 'q': 0}))



False


In [9]:
def esTautologia(formula: str) -> bool:
  """
  Verifica si una fórmula es una tautología o no
  """
  literales = set()
  for char in formula:
    if char.isalpha(): # Verificamos si es una letra
      literales.add(char)

  literales = sorted(list(literales)) # Lo convertimos en un conjunto ordenado
  len_lits = len(literales)

  for i in range(2**len_lits):
    binario = bin(i)[2:].zfill(len_lits) # Añade ceros hasta llegar a la longitud de la longitud de literales para evitar que haya menos valores que literales
    binario = [int(x) for x in binario]

    asignacion = dict(zip(literales, binario))
    valor = evaluarVerdad(formula, asignacion)

    print(f'Asignacion: {asignacion}, valor: {valor}')
    if valor == False:
      return False
  return True

formula_str = '(((p) * (!(p)))'
print(esTautologia(formula_str))

Asignacion: {'p': 0}, valor: True
Asignacion: {'p': 1}, valor: True
True


# Parte 3

In [3]:
def negar(literal: str):
  """
  Dado un literal niega este
  """
  if literal.startswith('!'):
    return literal[1]
  else:
    return '!' + literal

def es_tautologia(clausula: set):
  """
  Verifica si una cláusula es una tautología
  """
  for lit in clausula:
    if negar(lit) in clausula:
      return True
  return False

def resolver(clausula1: set, clausula2: set):
  """
  Dadas dos cláusulas intenta conseguir la resolvente de ellas,
  en caso de que no lo consiga o ésta sea una tautología devuelve None
  """
  # Volver cláusulas mutables
  c1 = set(clausula1)
  c2 = set(clausula2)

  for l in c1:
    l_negado = negar(l)

    if l_negado in c2:
      c1_resto = c1 - {l}
      c2_resto = c2 - {l_negado}
      resolvente = c1_resto | c2_resto

      if es_tautologia(resolvente):
        return None # No añadimos tautologías

      return resolvente # Realizamos este proceso una única vez para evitar eliminar más de un literal por clausula

  return None

def resolucion(lista: list) -> bool:
  """
  Dadas una lista de cláusulas, verifica si es una contradicción mediante el método de resolución
  """
  clausulas = set()
  for clausula in lista:
    if not es_tautologia(clausula):
      clausulas.add(frozenset(clausula)) # Los elementos de un set deben ser inmutables

  while True:
    generadas = set() # Cláusulas generadas en cada iteración

    clausulas_l = list(clausulas)

    for i, c1 in enumerate(clausulas_l):
      for c2 in clausulas_l[1+i:]:
        resolvente = resolver(c1, c2)

        if resolvente == set():
          return True # Es una contradicción

        if resolvente and frozenset(resolvente) not in clausulas:
          generadas.add(frozenset(resolvente))

    if not generadas:
      return False # No es contradicción

    clausulas.update(generadas)

  return False

c1 = {'p', '!u'}
c2 = {'p'}
c3 = {'t'}
c4 = {'q', '!s'}
c5 = {'s', '!r'}
c6 = {'r', '!p'}
c7 = {'!p', '!q', '!t'}
%timeit resolucion([c1, c2, c3, c4, c5, c6, c7])

502 µs ± 17.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Algoritmo SLD


In [10]:
def conversor_ch(lista):
  """
  Convierte una lista de conjuntos de literales (FNC) a una lista de
  cláusulas en formato [cabeza, cuerpo] para SLD.
  """
  clausulas_sld = []
  for clausula in lista:
    cabeza = set()
    cuerpo = set()
    for literal in sorted(clausula):
      if literal.startswith('!'):
        cuerpo.add(literal[-1])
      else:
        cabeza.add(literal)
    clausulas_sld.append([list(cabeza), list(cuerpo)])
  return clausulas_sld

def resolver(clausula1, clausula2):

  cabeza1, cuerpo1 = clausula1
  cabeza2, cuerpo2 = clausula2
  literal_eliminar = {cuerpo1[0]}

  return [list(set(cabeza1 + cabeza2).difference(literal_eliminar)), list(set(cuerpo1 + cuerpo2).difference(literal_eliminar))]

def algotirmo_sld(ob, clausulas_sld):

  cabeza_ob, cuerpo_ob = ob

  if not cabeza_ob and not cuerpo_ob:
    return True

  elif not cuerpo_ob:
    return False

  clausulas_p = []
  for clausula_kb in clausulas_sld:
    cabeza_kb, cuerpo_kb = clausula_kb
    if cuerpo_ob[0] in cabeza_kb:
      clausulas_p.append(clausula_kb)

  if not clausulas_p:
    return False

  for clausula_p in clausulas_p:
    nuevo_ob = resolver(ob, clausula_p)

    if algotirmo_sld(nuevo_ob, clausulas_sld):
      return True

  return False


def resolucion(lista):
  clausulas_sld = conversor_ch(lista)

  clausulas_obs = []
  if clausulas_sld == []:
    return True

  for clausula in clausulas_sld:
    if clausula[0] == []:
      clausulas_obs.append(clausula)

  if len(clausulas_obs) == 0:
    return False

  for clausula_ob in clausulas_obs:
    resultado = algotirmo_sld(clausula_ob, clausulas_sld)
    if resultado == True:
      return True
    else:
      continue

  return False


clausula1 = {'p', '!u'}
clausula2 = {'p'}
clausula3 = {'t'}
clausula4 = {'q', '!s'}
clausula5 = {'s', '!r'}
clausula6 = {'r', '!p'}
clausula7 = {'!p', '!q', '!t'}

resolucion([clausula1, clausula2, clausula3, clausula4, clausula5, clausula6, clausula7])

c1 = {'!t', 's'}
c2 = {'!p', 'q'}
c3 = {'!p', '!q', 'r'}
c4 = {'s', '!r'}
c5 = {'p'}
c6 = {'!s'}
resolucion([c1, c2, c3, c4, c5, c6])

True